In [1]:
from prehook import *
from hook import *
from lookups import *
from database_handler import *
import requests
from transformation_handler import *

In [2]:
execute_prehook()

Connecting to database "animal_intakes_outcomes"...
Connection was successful.
Closing connection to database...
Connection is closed.


In [3]:
execute_hook()

Connecting to database "animal_intakes_outcomes"...
Connection was successful.


c:\Users\olasm\Desktop\FSD - July 2023\week 9\FSD Final Project\database_handler.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_path)


Closing connection to database...
Connection is closed.


In [4]:
db_session = create_connection()

Connecting to database "animal_intakes_outcomes"...
Connection was successful.


In [5]:
query = "SELECT * FROM dw_reporting_schema.stg_intakes_outcomes_austin"
df = read_data_as_dataframe(InputTypes.SQL,query,db_session)
austin = df.copy()
austin.shape

c:\Users\olasm\Desktop\FSD - July 2023\week 9\FSD Final Project\database_handler.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_path)


(140249, 11)

In [53]:
norfolk['date_of_birth'] = norfolk.groupby('animal_id')['date_of_birth'].transform('first')

In [11]:
austin[austin.duplicated(subset='animal_id', keep=False)].sort_values('animal_id')

,animal_id,type,breed,color,intake_date,intake_type,sex,date_of_birth,outcome_date,outcome_type,region
287098,AUS-A006100,Dog,Spinone Italiano Mix,Yellow/White,2014-03-07 14:26:00,Public Assist,Neutered,2007-07-09,2014-03-08 17:10:00,Return to Owner,Austin
427335,AUS-A006100,Dog,Spinone Italiano Mix,Yellow/White,2014-03-07 14:26:00,Public Assist,Neutered,2007-07-09,2014-03-08 17:10:00,Return to Owner,Austin
146863,AUS-A006100,Dog,Spinone Italiano Mix,Yellow/White,2014-03-07 14:26:00,Public Assist,Neutered,2007-07-09,2014-03-08 17:10:00,Return to Owner,Austin
6680,AUS-A006100,Dog,Spinone Italiano Mix,Yellow/White,2014-03-07 14:26:00,Public Assist,Neutered,2007-07-09,2014-03-08 17:10:00,Return to Owner,Austin
288339,AUS-A047759,Dog,Dachshund,Tricolor,2014-04-02 15:55:00,Owner Surrender,Neutered,2004-04-02,2014-04-07 15:12:00,Transfer,Austin
...,...,...,...,...,...,...,...,...,...,...,...
560888,AUS-A891099,Cat,Domestic Shorthair,Black/Black,2023-10-17 13:27:00,Stray,nan,2262-04-11,2262-04-11 00:00:00,Pending,Austin
420653,AUS-A891099,Cat,Domestic Shorthair,Black/Black,2023-10-17 13:27:00,Stray,nan,2262-04-11,2262-04-11 00:00:00,Pending,Austin
280417,AUS-A891102,Cat,Domestic Shorthair,Black/Black,2023-10-17 13:27:00,Stray,nan,2262-04-11,2262-04-11 00:00:00,Pending,Austin
420652,AUS-A891102,Cat,Domestic Shorthair,Black/Black,2023-10-17 13:27:00,Stray,nan,2262-04-11,2262-04-11 00:00:00,Pending,Austin


In [24]:
duplicates = norfolk[norfolk.duplicated(subset='animal_id', keep=False)]
unique_duplicates = duplicates.drop_duplicates(subset='animal_id', keep='first')[['animal_id', 'date_of_birth']]
for index, row in unique_duplicates.iterrows():
    norfolk.loc[norfolk['animal_id'] == row['animal_id'], 'date_of_birth'] = row['date_of_birth']

In [25]:
query = "SELECT * FROM dw_reporting_schema.stg_intakes_outcomes_bloomington"
df = read_data_as_dataframe(InputTypes.SQL,query,db_session)
bloomington = df.copy()

c:\Users\olasm\Desktop\FSD - July 2023\week 9\FSD Final Project\database_handler.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_path)


In [44]:
bloomington[bloomington.duplicated(subset = 'id',keep='first')]
bloomington.drop_duplicates(subset=['id','intake_date'],keep='first',inplace = True)

In [45]:
bloomington.shape

(17164, 11)

In [9]:
close_connection(db_session)

Closing connection to database...
Connection is closed.


In [3]:
import lookups
import requests
import transformation_handler
import pandas as pd

In [4]:
url = lookups.DataSources.SHELTER_NORFOLK.value
response = requests.get(url,{'$limit':1000000})
df = pd.DataFrame(response.json())

In [5]:
dfs = {lookups.DataSources.SHELTER_NORFOLK.name.lower():df}

In [6]:
df_cleaned = transformation_handler.clean_norfolk_dataset(dfs)